In [4]:
import torch
import model5 as model
import os
from torch.utils.mobile_optimizer import optimize_for_mobile
from Tikki import Tikki
from player import AI

# Fuze
# Quantize

In [5]:
# Load model
n_players = 2
round_n = 5
card_n = 5
n_bets = 3
n_inputs = card_n + n_players*(1+round_n) + n_players
n_outputs = 52+3
hidden_size = 30
Actor = model.Actor(n_inputs,hidden_size,n_outputs)

model_folder_path = './model'
model_name = "actor_V1-2_lr6.08e-04,r4,lrbd0.90,g0.978591241041834,rbd0.97,hid30.pt"
file_name = os.path.join(model_folder_path,model_name)
loaded = torch.load(file_name)
Actor.load_state_dict(loaded["model_state"])
Actor.eval()

Actor(
  (embedding_layer): Embedding(72, 40)
  (hidden_layer1): Linear(in_features=760, out_features=30, bias=True)
  (hidden_layer2): Linear(in_features=30, out_features=30, bias=True)
  (hidden_layer3): Linear(in_features=30, out_features=30, bias=True)
  (output_layer): Linear(in_features=30, out_features=55, bias=True)
)

In [7]:
# Generate sample data
P1 = AI("P1",Actor,"cpu")
P2 = AI("P2",Actor,"cpu")
game = Tikki(players = [P1,P2])
game.new_game()
states  = []
for _ in range(10):
    while 1:
        # Or compute logprob, value directly with pass to net
        _, state, _, done = game.step(P1)
        states.append(state)

        if done:
            break
#states = torch.stack(states).squeeze()
traced_script_module = torch.jit.trace(Actor, state)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("model/mobile_model6.pt")

Bet probs: tensor([0., 1., 0.])
Bet probs: tensor([0., 1., 0.])
Card probs: tensor([1.1940e-16, 1.0000e+00, 2.7209e-18, 1.6474e-19, 3.3029e-17]))
Card probs: tensor([1.]))
Card probs: tensor([1.7919e-18, 1.0000e+00, 5.4006e-30, 2.8995e-23]))
Card probs: tensor([1.]))
Card probs: tensor([0.0727, 0.0021, 0.9253]))
Card probs: tensor([9.9913e-01, 2.9116e-05, 8.3793e-04]))
Card probs: tensor([0.9553, 0.0447]))
Card probs: tensor([0.5631, 0.4369]))
Card probs: tensor([1.]))
Card probs: tensor([1.]))
Bet probs: tensor([7.8140e-35, 1.0000e+00, 1.4013e-44])
Bet probs: tensor([0., 1., 0.])
Card probs: tensor([7.3514e-01, 1.5264e-04, 6.4580e-02, 2.0007e-01, 5.7259e-05]))
Card probs: tensor([1.]))
Card probs: tensor([1.3558e-04, 2.0309e-05, 5.8317e-04, 9.9926e-01]))
Card probs: tensor([1.]))
Card probs: tensor([0.1113, 0.0314, 0.8572]))
Card probs: tensor([1.]))
Card probs: tensor([0.5332, 0.4668]))
Card probs: tensor([0.6854, 0.3146]))
Card probs: tensor([1.]))
Card probs: tensor([1.]))
Bet prob

In [8]:
traced_script_module.save("model/model6.pt")